In [1]:
import pandas as pd
import geopandas as gpd
import datetime
from shapely.geometry import Point
import os
import json
import csv
import numpy as np
from shapely.geometry import Point
import time
from dateutil.parser import parse

In [2]:
rest = pd.read_csv('lateNightRest.csv')
rest.head()

,restaurant_alias,restaurant_categories_0_alias,restaurant_categories_0_title,restaurant_categories_1_alias,restaurant_categories_1_title,restaurant_categories_2_alias,restaurant_categories_2_title,restaurant_coordinates_latitude,restaurant_coordinates_longitude,restaurant_display_phone,...,restaurant_location_zip_code,restaurant_name,restaurant_phone,restaurant_price,restaurant_rating,restaurant_review_count,restaurant_transactions_0,restaurant_transactions_1,restaurant_transactions_2,restaurant_url
0,the-halal-guys-new-york-2,foodstands,Food Stands,mideastern,Middle Eastern,halal,Halal,40.761756,-73.979230,NaN,...,10019.0,The Halal Guys,NaN,$,4.0,9029,NaN,NaN,NaN,https://www.yelp.com/biz/the-halal-guys-new-yo...
1,late-nite-hackensack-pizza-and-steak-hackensack,pizza,Pizza,tradamerican,American (Traditional),italian,Italian,40.900701,-74.041477,(201) 342-5283,...,7601.0,Late Nite Hackensack Pizza and Steak,1.201343e+10,$$,4.0,78,NaN,NaN,NaN,https://www.yelp.com/biz/late-nite-hackensack-...
2,buddakan-new-york,chinese,Chinese,bars,Bars,asianfusion,Asian Fusion,40.742276,-74.004800,(212) 989-6699,...,10011.0,Buddakan,1.212990e+10,$$$,4.0,3599,pickup,delivery,NaN,https://www.yelp.com/biz/buddakan-new-york?adj...
3,turntable-chicken-jazz-new-york-2,chicken_wings,Chicken Wings,korean,Korean,NaN,NaN,40.748280,-73.986490,(212) 714-9700,...,10001.0,Turntable Chicken Jazz,1.212715e+10,$$,4.0,1793,NaN,NaN,NaN,https://www.yelp.com/biz/turntable-chicken-jaz...
4,the-meatball-shop-new-york,italian,Italian,tradamerican,American (Traditional),NaN,NaN,40.721496,-73.988736,(212) 982-8895,...,10002.0,The Meatball Shop,1.212983e+10,$$,4.0,2684,pickup,delivery,NaN,https://www.yelp.com/biz/the-meatball-shop-new...


In [3]:
rest = rest[['restaurant_name', 'restaurant_location_state', 'restaurant_location_zip_code', 'restaurant_location_city',
             'restaurant_coordinates_latitude', 'restaurant_coordinates_longitude', 'restaurant_rating', 'restaurant_price']]
rest = rest[(rest.restaurant_location_state == 'NY')]
rest = rest[(rest.restaurant_location_city == 'New York')]
rest['lonlat']=list(zip(rest.restaurant_coordinates_longitude,rest.restaurant_coordinates_longitude))
geometry = [Point(xy) for xy in zip(rest.restaurant_coordinates_longitude, rest.restaurant_coordinates_latitude)]
crs = {'init': 'epsg:4326'}
rest = gpd.GeoDataFrame(rest, crs=crs, geometry=geometry)
rest.head()

,restaurant_name,restaurant_location_state,restaurant_location_zip_code,restaurant_location_city,restaurant_coordinates_latitude,restaurant_coordinates_longitude,restaurant_rating,restaurant_price,lonlat,geometry
0,The Halal Guys,NY,10019.0,New York,40.761756,-73.979230,4.0,$,"(-73.9792299271, -73.9792299271)",POINT (-73.97922992709999 40.7617559583)
2,Buddakan,NY,10011.0,New York,40.742276,-74.004800,4.0,$$$,"(-74.0048000962, -74.0048000962)",POINT (-74.0048000962 40.7422762672)
3,Turntable Chicken Jazz,NY,10001.0,New York,40.748280,-73.986490,4.0,$$,"(-73.98649, -73.98649)",POINT (-73.98649 40.74828)
4,The Meatball Shop,NY,10002.0,New York,40.721496,-73.988736,4.0,$$,"(-73.9887357, -73.9887357)",POINT (-73.98873570000001 40.7214961)
5,Great NY Noodle Town,NY,10013.0,New York,40.715036,-73.997117,3.5,$,"(-73.9971172437, -73.9971172437)",POINT (-73.9971172437 40.7150363829)


In [4]:
rest['hashtag'] = rest['restaurant_name'].str.replace(" ", "")
rest['hashtags'] = rest['hashtag'].str.lower()
rest.head()

,restaurant_name,restaurant_location_state,restaurant_location_zip_code,restaurant_location_city,restaurant_coordinates_latitude,restaurant_coordinates_longitude,restaurant_rating,restaurant_price,lonlat,geometry,hashtag,hashtags
0,The Halal Guys,NY,10019.0,New York,40.761756,-73.979230,4.0,$,"(-73.9792299271, -73.9792299271)",POINT (-73.97922992709999 40.7617559583),TheHalalGuys,thehalalguys
2,Buddakan,NY,10011.0,New York,40.742276,-74.004800,4.0,$$$,"(-74.0048000962, -74.0048000962)",POINT (-74.0048000962 40.7422762672),Buddakan,buddakan
3,Turntable Chicken Jazz,NY,10001.0,New York,40.748280,-73.986490,4.0,$$,"(-73.98649, -73.98649)",POINT (-73.98649 40.74828),TurntableChickenJazz,turntablechickenjazz
4,The Meatball Shop,NY,10002.0,New York,40.721496,-73.988736,4.0,$$,"(-73.9887357, -73.9887357)",POINT (-73.98873570000001 40.7214961),TheMeatballShop,themeatballshop
5,Great NY Noodle Town,NY,10013.0,New York,40.715036,-73.997117,3.5,$,"(-73.9971172437, -73.9971172437)",POINT (-73.9971172437 40.7150363829),GreatNYNoodleTown,greatnynoodletown


In [12]:
rest55050 = rest.loc[550:600]
rest55050.head()

,restaurant_name,restaurant_location_state,restaurant_location_zip_code,restaurant_location_city,restaurant_coordinates_latitude,restaurant_coordinates_longitude,restaurant_rating,restaurant_price,lonlat,geometry,hashtag,hashtags
550,Ningbo Cafe,NY,10280.0,New York,40.708140,-74.017790,4.0,$$,"(-74.01779, -74.01779)",POINT (-74.01779000000001 40.70814),NingboCafe,ningbocafe
551,Two Lizards Mexican Bar & Grill,NY,10021.0,New York,40.768857,-73.955524,3.5,$$,"(-73.955524, -73.955524)",POINT (-73.955524 40.768857),TwoLizardsMexicanBar&Grill,twolizardsmexicanbar&grill
552,Mamasita Bar & Grill,NY,10019.0,New York,40.767330,-73.989450,4.0,$$,"(-73.98945, -73.98945)",POINT (-73.98945000000001 40.76733),MamasitaBar&Grill,mamasitabar&grill
553,Emmett's,NY,10012.0,New York,40.727303,-74.002430,4.0,$$,"(-74.00243, -74.00243)",POINT (-74.00243 40.72730300000001),Emmett's,emmett's
554,Stax Sports Bar,NY,10075.0,New York,40.772310,-73.956030,4.5,$$,"(-73.95603, -73.95603)",POINT (-73.95603 40.77231),StaxSportsBar,staxsportsbar


In [13]:
rest55050Times = {}
for hashtags in rest55050['hashtags']:
    rest55050Times[str(hashtags)] = []
    
rest55050Times

{'alphabetlounge': [],
 'americancutsteakhousetribeca': [],
 'bacaro': [],
 'barbalurestaurant': [],
 "benny'sburritos": [],
 'cafedenovo': [],
 'calexico': [],
 'ciboevino': [],
 'cieloatthemayfair': [],
 'clubcumming': [],
 'crispypizza': [],
 'daclaudio': [],
 'dagennaro': [],
 'delacortetheater': [],
 "emmett's": [],
 'flatironlounge': [],
 'flipsigi': [],
 "gabriel'sbar&restaurant": [],
 'gradisca': [],
 "harry's": [],
 'ilbucoalimentari&vineria': [],
 'jimmy': [],
 'kindregards': [],
 'lezie': [],
 'librarybarathudson': [],
 'losfeliz': [],
 'losmariscos': [],
 'mamasitabar&grill': [],
 'ningbocafe': [],
 'obao': [],
 'osteriacotta': [],
 'pugliaristorante': [],
 'roccosteakhouse': [],
 'ruma&shanta': [],
 'sfogliarestaurant': [],
 'shanghaimong': [],
 'sohocigarbar': [],
 'staxsportsbar': [],
 'taqueriast.marksplace': [],
 'thewritingroom': [],
 'twolizardsmexicanbar&grill': [],
 "victor'scafe": []}

In [14]:
for hashtag in rest55050Times.keys():
    try:
        !eval {"python app.py {tag} --tag -u dildo921 -p Horror@10 -m 1000".format(tag=hashtag)}
    
    finally:
        try:
            data = pd.read_csv('newCsv1.csv')
            for i in range(len(data)):
    
                y = time.ctime(data.values[i])
                z = parse(y)
                rest55050Times[hashtag].append(z)
            os.remove('newCsv1.csv')
        except IOError:
                pass

Searching jimmy for posts: 961 media [04:50,  3.07s/ media]
Downloading: 100%|██████████| 1000/1000 [00:00<00:00, 34302.78it/s]/bin/sh: line 1:  2031 Segmentation fault: 11  python app.py jimmy --tag -u dildo921 -p Horror@10 -m 1000
Searching delacortetheater for posts: 899 media [04:51,  6.00s/ media]
Downloading:   0%|          | 0/1000 [00:00<?, ?it/s]
Searching lezie for posts: 900 media [14:12,  5.69s/ media].13it/s]
Downloading:   0%|          | 0/1000 [00:00<?, ?it/s]
Searching americancutsteakhousetribeca for posts: 0 media [00:00, ? media/s]
/bin/sh: shanta: command not found
Searching ningbocafe for posts: 39 media [00:11,  3.54 media/s]
Downloading: 100%|██████████| 39/39 [00:00<00:00, 4397.25it/s]
Searching osteriacotta for posts: 256 media [10:45,  2.52s/ media] 
Downloading: 100%|██████████| 256/256 [00:00<00:00, 8578.27it/s]
Searching flatironlounge for posts: 443 media [01:45,  4.19 media/s]
Downloading: 100%|██████████| 443/443 [00:00<00:00, 18428.36it/s]
Searching bac

In [15]:
rest55050Times

{'alphabetlounge': [datetime.datetime(2016, 8, 25, 9, 44, 40),
  datetime.datetime(2016, 8, 24, 11, 3, 16),
  datetime.datetime(2017, 10, 8, 17, 52, 15),
  datetime.datetime(2017, 10, 5, 15, 16, 37),
  datetime.datetime(2017, 5, 13, 17, 10, 27),
  datetime.datetime(2017, 10, 3, 23, 42, 6),
  datetime.datetime(2018, 11, 7, 14, 3, 28),
  datetime.datetime(2018, 5, 22, 20, 7, 49),
  datetime.datetime(2018, 3, 23, 18, 29, 4),
  datetime.datetime(2018, 1, 29, 19, 24, 53),
  datetime.datetime(2017, 12, 29, 13, 45, 17),
  datetime.datetime(2017, 12, 26, 13, 31, 57),
  datetime.datetime(2017, 12, 18, 15, 29, 44),
  datetime.datetime(2017, 12, 25, 9, 2, 14),
  datetime.datetime(2017, 12, 23, 16, 11, 45),
  datetime.datetime(2017, 12, 21, 19, 21, 46),
  datetime.datetime(2017, 12, 9, 11, 38, 56),
  datetime.datetime(2017, 12, 8, 12, 53, 20),
  datetime.datetime(2017, 12, 7, 23, 58, 20),
  datetime.datetime(2017, 12, 7, 9, 57, 31),
  datetime.datetime(2017, 11, 24, 10, 30, 51),
  datetime.datetim

In [16]:
np.save("data55050.npy", rest55050Times)

In [17]:
x = np.load('data35050.npy')

In [20]:
print x.item().get('nyonya')[0]

2018-07-08 07:31:18


In [25]:
for keys in x.item():
    print keys

hunt&fishclub
granmorsi
réunion
briciola
pazzanotte
revisionlounge&gallery
eltoroblanco
javelina
take31
manhattandiner
luckyjack's
porsena
calico'snyc
lincolnsquaresteak
fondaeastvillage
plungerooftopbar+lounge
eataly
kanghodongbaekjeong
freshsalt
vinsurvingt
ilbuco
donsbogambbq&winebar
rabbithouse
giorgio'sofgramercy
hangawi
49monroe
kestèwallstreet
reservecut
spaghettiincident
hornblowercruises&events-pier40
whispersrestaurant,bar&lounge
cibar
john'softimessquare
marqueenightclub
brooklynchophouse
royale
pisticci
parksidelounge
bastapasta
rosamexicano
automaticslims
piacere
tacombi-nolita
nyonya
zengo
burgerjoint
wahfungno1fastfood
piccolastrada
